## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [ ]:
# poetry add langchain-tavily
# 최신 langchain 0.3.25 최신버전 기준  langchain-tavily 포함된 클래스를 사용
from langchain_tavily.tavily_search import TavilySearch

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearch(max_results=3)

# 웹 검색 실행
search_results = web_search.invoke(query)
print(type(search_results))
#pprint(search_results)

# 검색 결과 출력
for result in search_results['results']:
    print(type(result))
    pprint(result)  
    print("-" * 100)  

In [ ]:
# 최신 langchain 0.3.25 이전 버전 langchain community 에 포함된 클래스를 사용
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=3)

# 웹 검색 실행
search_results = web_search.invoke(query)
print(type(search_results))
#pprint(search_results)

# 검색 결과 출력
for result in search_results:
    print(type(result))
    pprint(result)  
    print("-" * 100)  

In [32]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [45]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

<class 'langchain_core.runnables.base.RunnableBinding'>


In [17]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)
print(type(ai_msg))

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BisvEwZw8l2z6tL1c1xIzv3uTTPaj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e4c4114-ae21-4c55-904d-67fd87f5b497-0', usage_metadata={'input_tokens': 87, 'output_tokens': 22, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
--------------------------

In [ ]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
#pprint(dir(ai_msg))

In [20]:

pprint(vars(ai_msg))

{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query":"wines '
                                                                 'that pair '
                                                                 'well with '
                                                                 'steak"}',
                                                    'name': 'tavily_search_results_json'},
                                       'id': 'call_dk67HueDxXBX1mj1YCdEV33G',
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run--1e399d70-44d3-4924-9153-9746e821b4bd-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-BisxAAm84UoHFgUZbczPZW5Zn952F',
                       'logprobs': None,
                       'model_name': 'gpt-3.5-turbo-0125',
                       'service_tier': 'default',
       

In [35]:

pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WF8RGHoChNjDEq0BvL4GnrPS', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 91, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bit9uYdz9cx4iCWt2NHecANlqbGtf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b78ee6bc-7496-440d-a043-1fccb1736296-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_WF8RGHoChNjDEq0BvL4GnrPS', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 

In [36]:
# AIMessage 객체에 포함된 
tool_call = ai_msg.tool_calls[0]

pprint(tool_call)

{'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_WF8RGHoChNjDEq0BvL4GnrPS',
 'name': 'tavily_search_results_json',
 'type': 'tool_call'}


`(3) 도구(tool) 실행하기`

#### 방법 1: 직접 도구 호출 처리한 결과로 ToolMessage 객체 생성

* 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
* 'args' 속성을 사용하여 TavilySearch 객체의 invoke() 호출하여 결과를 얻는다.


In [37]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n'
             '------------------------------------------------\n'
             '\n'
             '![Image '
             '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
             '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n'
             '\n'
             '**추천 이유:**\n'
             '\n'
             '카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 '
             '이루며, 고기의 감칠맛을 더욱 살려줍니다.\n'
             '\n'
             '**어울리는 스테이크:**\n'
             '\n'
             '*   리브아이 스테이크\n'
             '*   뉴욕 스트립\n'
             '*   티본 스테이크\n'
             '\n'
             '**추천 와인:**\n'
             '\n'
             '*   나파 밸리 카베르네 소비뇽\n'
             '*   샤또 마고 (Château Margaux) [...] !

In [38]:

from langchain_core.messages import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

pprint(tool_message.content)

[{'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n'
             '------------------------------------------------\n'
             '\n'
             '![Image '
             '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
             '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n'
             '\n'
             '**추천 이유:**\n'
             '\n'
             '카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 '
             '이루며, 고기의 감칠맛을 더욱 살려줍니다.\n'
             '\n'
             '**어울리는 스테이크:**\n'
             '\n'
             '*   리브아이 스테이크\n'
             '*   뉴욕 스트립\n'
             '*   티본 스테이크\n'
             '\n'
             '**추천 와인:**\n'
             '\n'
             '*   나파 밸리 카베르네 소비뇽\n'
             '*   샤또 마고 (Château Margaux) [...] ![Image '
             '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
             '11](h

#### 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성
```
'tool_calls': [{'args': {'query': 'wines that pair well with steak'},
                 'id': 'call_dk67HueDxXBX1mj1YCdEV33G',
                 'name': 'tavily_search_results_json',
                 'type': 'tool_call'}],
```
* tool_call 변수는 AIMessage 객체 포함된 tool 을 호출한 결과 tool_calls의 첫번째 dict 객체
* TavilySearch 객체를 invoke(tool_call) 하면 결과객체의 타입이 ToolMessage 이다.
* 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
* 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.


In [39]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")


<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: [{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)!

In [40]:
pprint(tool_message.content)


('[{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 '
 '대표주자\\n------------------------------------------------\\n\\n![Image '
 '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
 '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 '
 '이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, '
 '고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   '
 '티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château '
 'Margaux) [...] ![Image '
 '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
 '11](h

In [41]:
pprint(tool_message.tool_call_id)

'call_WF8RGHoChNjDEq0BvL4GnrPS'


In [42]:
pprint(tool_message.name)

'tavily_search_results_json'


In [43]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '스테이크와 어울리는 와인 추천'},
  'id': 'call_WF8RGHoChNjDEq0BvL4GnrPS',
  'type': 'tool_call'}]

In [44]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

* 파라미터 이름이 config여야 하는 이유: @chain 데코레이터가 config라는 이름만 특별히 인식하도록 설계되었습니다.
* config 생략 가능한 이유: @chain이 자동으로 기본 RunnableConfig()를 생성해서 전달하기 때문입니다.

In [47]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")
print(type(llm))
# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
tavily_tool = TavilySearchResults(max_results=2)
print(type(tavily_tool))

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])
print(type(llm_with_tools))
#<class 'langchain_core.runnables.base.RunnableBinding'>

# LLM 체인 생성
llm_chain = prompt | llm_with_tools
print(type(llm_chain))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
<class 'langchain_core.runnables.base.RunnableBinding'>
<class 'langchain_core.runnables.base.RunnableSequence'>


In [48]:

# 도구 실행 체인 정의
# @chain 은 일반 함수를 Runnable 객체로 만들어주는 데코레이터 
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print(type(ai_msg))
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print(type(tool_msgs))
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_IIjVgiRCGD1bGU74z2liCiUr', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_Hv7v2Vr0iWW8gYypX8yItJND', 'function': {'arguments': '{"query": "스테이크에 잘 맞는 와인 종류"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 109, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bitfcs62KA7FBT4p5ZYHOohiQRqC7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b0a146cd-1efb-4dd0-87c8-14e304b9fb8d-0' t

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [56]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

- wikipedia 설치
```
    poetry add wikipedia
```
- [WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html) 사용


In [57]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)
print(type(wiki_search))

<class 'langchain_core.tools.structured.StructuredTool'>


In [ ]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

In [ ]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(type(result))
    print(result)  
    print("-" * 100)  

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
print(type(tavily_search_func)) # @tool 선언
print(type(wiki_search)) # as_tool() 함수사용

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.tools.structured.StructuredTool'>
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2JVbyk6KE5AiRCk5XDtU9D8B', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_za8eSproR8w9k4VV1l1kvEMs', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 176, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BivgvVWc4HTibpQ32djosRPfruNQF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c548382

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 오스트레일리아의 투움바에서 이름을 따왔으며, 매콤한 크림 소스를 사용한 '
 '두꺼운 파스타이다. 한국과 브라질에서 인기를 끌고 있으며, 다양한 변형 요리도 존재한다. 피치는 이탈리아의 전통 파스타로, 손으로 '
 '만들어지는 굵은 형태로, 주로 밀가루와 물로 반죽하여 만든다. 주로 육수와 함께 요리되어 다양한 방식으로 즐길 수 있다.')


In [60]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [61]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5qS0vidLT8IdkPGcwxse8i8l', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_3E7XGWEczdlGfXMIMlAmEy3O', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 152, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BivmY9DIYLOb7BQ0OW9TKQ168rMAG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--82d3ef97-dbb6-4c2e-b855-65405153d07a-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, '

In [62]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타의 유래'},
 'id': 'call_3E7XGWEczdlGfXMIMlAmEy3O',
 'type': 'tool_call'}

In [63]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])
print(type(tool_message))
print(tool_message)
print("-" * 100)
pprint(tool_message.content)

<class 'langchain_core.messages.tool.ToolMessage'>
content='투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 한국과 브라질에서 인기를 끌고 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 있으며 북부는 쌀과 유제품, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.' name='wiki_summary' tool_call_id='call_3E7XGWEczdlGfXMIMlAmEy3O'
----------------------------------------------------------------------------------------------------
('투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 '
 '한국과 브라질에서 인기를 끌고 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 이탈리아 요리는 기원전 4세기부터 '
 '발전해온 남유럽의 요리로, 지역마다 특색이 있으며 북부는 쌀과 유제품, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 '
 '재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.')


In [64]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_UmOfLWjhiAysnJsxGpiD21av', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BivqfsCG1cJFprtuz5iU8lxVz7QKu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--427ba0e3-2112-4e51-a74a-13b7151e8ef5-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_UmOfLWjhiAysnJsxGpiD21av', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [65]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [71]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '시그니처 스테이크',
              'menu_number': 1,
              'source': './data/restaurant_menu.txt'},
 'page_content': '1. 시그니처 스테이크\n'
                 '   • 가격: ₩35,000\n'
                 '   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n'
                 '   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 '
                 '레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. '
                 '레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '트러플 리조또',
              'menu_number': 2,
              'source': './data/restaurant_menu.txt'},
 'page_content': '2. 트러플 리조또\n'
                 '   • 가격: ₩22,000\n'
                 '   • 주요 식재료

### Embedding을 하기 위해 Ollama Qwen2.5 설치
```
ollama run qwen2.5:1.5b

ollama ls
```

In [82]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 4},
)


In [83]:

# 쿼리 테스트
query = "스테이크 메뉴의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 4개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 8
메뉴 이름: 안심 스테이크 샐러드

메뉴 번호: 6
메뉴 이름: 해산물 파스타

메뉴 번호: 7
메뉴 이름: 랍스터 비스크



- 와인 메뉴에 대해서도 같은 작업을 처리

In [86]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

embeddings_model = OllamaEmbeddings(model="bge-m3:latest")

# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

wine_db.save_local("./db/wine_db")

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 4},
)


총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...


In [88]:

query = "주요 품종이 카베르네 소비뇽 인 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 4개
메뉴 번호: 8
메뉴 이름: 오퍼스 원 2017

메뉴 번호: 4
메뉴 이름: 클로 뒤 발 2016

메뉴 번호: 3
메뉴 이름: 사시카이아 2018

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015



`(2) 도구(tool) 정의하기`

In [89]:
from langchain_core.tools import tool

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_menu_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_menu_func',
 'type': 'object'}


In [90]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_wine_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_wine_func',
 'type': 'object'}


In [91]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크의 가격과 특징은 무엇인가요? 주요 품종이 카베르네 소비뇽 인 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AHdAdsO5wXQJdjHyykzvbLLK', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'db_search_menu_func'}, 'type': 'function'}, {'id': 'call_9JqO76nIw9lWIdknjljADa8e', 'function': {'arguments': '{"query": "카베르네 소비뇽"}', 'name': 'db_search_wine_func'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 140, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Biwa2TEyXqF7TsUMpa9MKksVQosQp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c0899729-7979-4fb8-9929-5e3960b38880-0', tool_calls=[{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': '

`(3) 여러 개의 도구(tool) 호출하기`

In [92]:
# @tool : tavily_search_func,db_search_menu_func,db_search_wine_func   
# as_tool()함수 : wiki_summary, 
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(type(tool))
    print(tool.name)

<class 'langchain_core.tools.structured.StructuredTool'>
tavily_search_func
<class 'langchain_core.tools.structured.StructuredTool'>
wiki_summary
<class 'langchain_core.tools.structured.StructuredTool'>
db_search_menu_func
<class 'langchain_core.tools.structured.StructuredTool'>
db_search_wine_func


In [118]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("%"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n") 
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [121]:

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 해주세요.")

# 응답 출력 
print(response.content)

('db_search_menu_func: \n'
 "{'name': 'db_search_menu_func', 'args': {'query': '시그니처 스테이크'}, 'id': "
 "'call_5nAvlBSKkdXFegjAf9oCTSIw', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('db_search_wine_func: \n'
 "{'name': 'db_search_wine_func', 'args': {'query': '시그니처 스테이크'}, 'id': "
 "'call_bMyZyOUaE8vkRMHhJ7m01QaE', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_menu_func'
'db_search_wine_func'
----------------------------------------------------------------------------------------------------
### 시그니처 스테이크

- **가격**: ₩35,000
- **특징**: 시그니처 스테이크는 고급 한우 등심으로, 로즈마리를 곁들인 소스와 특별히 조리된 리드미컬한 완성도가 특징입니다. 이 스테이크는 담백하게 조리되어 고기의 자연스러운 맛을 살려주며, 부드러운 식감을 자랑합니다. 미디엄 레어로 조리하여 육즙이 가득한 맛을 느낄 수 있습니다. 

### 추천 와인

1. **그라지 2016**
   - **가격**: ₩950,000
   - **특징**: 그라지 2016은 혼합형 레드 와인으로, 향이 깊고 복합적인 맛을 자랑합니다. 블랙베리, 자두의 풍미가 선명하게 느

In [122]:
print(type(response))
pprint(vars(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
{'additional_kwargs': {'refusal': None},
 'content': '### 시그니처 스테이크\n'
            '\n'
            '- **가격**: ₩35,000\n'
            '- **특징**: 시그니처 스테이크는 고급 한우 등심으로, 로즈마리를 곁들인 소스와 특별히 조리된 리드미컬한 완성도가 '
            '특징입니다. 이 스테이크는 담백하게 조리되어 고기의 자연스러운 맛을 살려주며, 부드러운 식감을 자랑합니다. 미디엄 '
            '레어로 조리하여 육즙이 가득한 맛을 느낄 수 있습니다. \n'
            '\n'
            '### 추천 와인\n'
            '\n'
            '1. **그라지 2016**\n'
            '   - **가격**: ₩950,000\n'
            '   - **특징**: 그라지 2016은 혼합형 레드 와인으로, 향이 깊고 복합적인 맛을 자랑합니다. 블랙베리, '
            '자두의 풍미가 선명하게 느껴지며, 부드러운 타닌과 긴 여운이 특징입니다. 각종 육류와 잘 어울리며, 특히 스테이크와 '
            '짝을 이룰 때 최상의 조화를 이룹니다.\n'
            '\n'
            '2. **오르페스 원 2017**\n'
            '   - **가격**: ₩650,000\n'
            '   - **특징**: 오르페스 원 2017은 프랑스산 레드 와인으로, 카베르네 소비뇽과 메를로 블렌드입니다. 과일의 '
            '향과 스파이스, 섬세한 산미가 잘 조화되어 있으며, 덕분에 풍부한 맛의 스테이크와 훌륭한 페어링을 제공합니다.\n'
            '\n'
            '이 두 가지 와인은 시그니처 스테

In [120]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

('db_search_menu_func: \n'
 "{'name': 'db_search_menu_func', 'args': {'query': '파스타'}, 'id': "
 "'call_iuFvUSU5r82G471rHVSdnuzM', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('wiki_summary: \n'
 "{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': "
 "'call_G1TIm01Ak4ofmI9y8cVD72wz', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_menu_func'
'wiki_summary'
----------------------------------------------------------------------------------------------------
### 파스타 메뉴
특정 식당의 파스타 메뉴는 다음과 같습니다:

1. **해산물 파스타**
   - 가격: ₩24,000
   - 재료: 링귀니 파스타, 새우, 문어, 오징어, 토마토 소스
   - 설명: 살아있는 해산물을 넉넉히 넣어 만들어진 해산물의 깊은 맛을 더한 파스타입니다. 토마토 소스의 상큼함과 해산물의 감칠맛이 조화를 이룹니다.

2. **뇨끼 파스타**
   - 가격: ₩28,000
   - 재료: 뇨끼, 스위트콘, 베이컨, 파르메산
   - 설명: 뇨끼의 쫄깃함과 스위트콘의 단맛, 베이컨의 풍미가 어우러진 맛있는 파스타입니다.

3. **트러플 파스타**
   - 가격: ₩9,000
   - 재료: 마늘파스타,

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [123]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the db_search_menu_func tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the db_search_wine_func tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_87x2AWjtoHloyTu7o1DiQgIC', 'type': 'tool_call'}
{'name': 'db_search_wine_func', 'args': {'query': '스테이크에 어울리는 와인'}, 'id': 'call_qWTXXl9U5cakexoftOOqNQPK', 'type': 'tool_call'}


In [124]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타 유래'}, 'id': 'call_TNBQUSS7FGVbop1n51CypoOb', 'type': 'tool_call'}
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_ailB56hp4kmQTiwMks7fdiDY', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [125]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the db_search_menu_func tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the db_search_wine_func tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools


In [ ]:

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

In [124]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_P0qIdjdxdMrkuApJ7QBFkIxu', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '강남 파스타 맛집'}, 'id': 'call_Jah4F00UAaYQVJrAvToHAIWM', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 단종되었고, 한국과 브라질에서 여전히 인기 있는 메뉴로 남아 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 오르조는 이탈리아의 파스타로, 보리 모양을 하고 있으며 주로 수프와 함께 먹는다. 오르조는 다양한 이름으로 불리며, 터키에서는 아르파 셰흐리예로 알려져 있다.', name='wiki_summary', tool_call_id='call_P0qIdjdxdMrkuApJ7QBFkIxu'), ToolMessage(content="[Document(id='4bedbd11-6a57-4eca-9a3c-fdf5af20abf6', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 3, 'menu_name': '연어 타르타르'}, page_content

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [125]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [126]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [127]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `db_search_menu_func` with `{'query': '시그니처 스테이크'}`


[Document(id='bc6ccf68-2fdb-447f-bd61-7a807c1c6a5c', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='6902c152-381f-4254-a7fb-8394eec0d9fa', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 2, 'menu_name': '트러플 리조또'}, page_content='2. 트러플 리조또\n   • 가격: ₩22,000\n   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈\n   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.')]
Invoking: `db_search_wine_func` with `{'query': '스테이크'}`


[Document(id='601e6f15-4d58-427b-a070-a7161782c2f9', metadata={'sou

In [128]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '### 시그니처 스테이크 정보\n'
           '\n'
           '- **가격**: ₩35,000\n'
           '- **주재료**: 소고기, 로즈마리, 허브 믹스\n'
           '- **특징**: 신선한 질 좋은 소고기를 사용하여 은은한 로즈마리 향과 허브 믹스를 곁들여 조리합니다. 반숙 달걀과 '
           '함께 제공되어 특별한 맛을 더합니다. 특히, 스테이크의 식감과 맛을 최대로 느낄 수 있도록 조리되어 있습니다.\n'
           '\n'
           '### 스테이크 와인 추천\n'
           '\n'
           '1. **포르트 와인 1er Cru 2018**\n'
           '   - **가격**: ₩320,000\n'
           '   - **주요 특징**: 진한 향미와 다양한 과일의 맛을 가지고 있어 스테이크와의 조화가 뛰어납니다.\n'
           '  \n'
           '2. **그릇지 2016**\n'
           '   - **가격**: ₩950,000\n'
           '   - **주요 특징**: 복합적인 맛과 풍부한 향이 특징이며, 스테이크의 풍미를 더욱 돋보이게 만들어 줍니다.\n'
           '\n'
           '스테이크와 잘 어울리는 와인 중 포르트 와인은 그 자체로도 풍미가 깊어, 고기의 짭짤함과 좋은 균형을 이룹니다. 반면 '
           '그릇지는 더욱 고급스러운 경험을 제공하여 특별한 날에 잘 어울립니다. 어떤 선택을 하시든 멋진 식사를 즐기실 수 '
           '있습니다!'}


## 5. Gradio 활용

In [129]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `db_search_wine_func` with `{'query': '트러플 리조또'}`


[Document(id='389dc8a9-475e-4d1a-9409-a033904537ab', metadata={'source': './data/restaurant_wine.txt', 'menu_number': 10, 'menu_name': '그랜지 2016'}, page_content='10. 그랜지 2016\n    • 가격: ₩950,000\n    • 주요 품종: 시라\n    • 설명: 호주의 대표적인 아이콘 와인입니다. 블랙베리, 자두, 블랙 올리브의 강렬한 과실향과 함께 유칼립투스, 초콜릿, 가죽의 복잡한 향이 어우러집니다. 풀바디이며 강렬한 타닌과 산도가 특징적입니다. 놀라운 집중도와 깊이, 긴 여운을 자랑하며, 수십 년의 숙성 잠재력을 가집니다.'), Document(id='601e6f15-4d58-427b-a070-a7161782c2f9', metadata={'source': './data/restaurant_wine.txt', 'menu_number': 7, 'menu_name': '풀리니 몽라쉐 1er Cru 2018'}, page_content='7. 풀리니 몽라쉐 1er Cru 2018\n   • 가격: ₩320,000\n   • 주요 품종: 샤르도네\n   • 설명: 부르고뉴 최고의 화이트 와인 중 하나로 꼽힙니다. 레몬, 사과, 배의 과실향과 함께 헤이즐넛, 버터, 바닐라의 풍부한 향이 어우러집니다. 미네랄리티가 돋보이며, 크리미한 텍스처와 긴 여운이 특징입니다. 해산물, 닭고기, 크림 소스 파스타와 좋은 페어링을 이룹니다.')]트러플 리조또와 잘 어울리는 와인으로는 **풀리니 몽라쉐 1er Cru 2018**를 추천드립니다. 가격은 ₩320,000입니다.

이 와인은 부르고뉴 최고의 화이트 와인 중 하나로, 레몬, 사과, 배의 과일 향이 돋보이며, 

In [55]:
# 데모 종료
demo.close()

Closing server running on port: 7860
